In [2729]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

## Read the data

In [2730]:
# rfr file, rm file
file_name_rfr = r"..\..\Sray\original dirty data\New_rfr.xlsx"
file_name_rm = r"..\..\Sray\original dirty data\New_rm.xlsx"

#get the sorted fin file

# file_name_str = r"..\..\Sray\Analysis\AU\AU_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\..\Sray\Analysis\CN\CN_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\..\Sray\Analysis\HK\HK_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\..\Sray\Analysis\MY\MY_Bloomberg_fin_data_portfolio.xlsx"
file_name_str = r"..\..\Sray\Analysis\SG\SG_Bloomberg_fin_data_portfolio.xlsx"



# indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'historical_market_cap'
# indicator = 'roa'
indicator = 'vol_180d'
## indicator = 'return_com_eqy'

file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)
rfr_file = pd.ExcelFile(file_name_rfr)
rm_file = pd.ExcelFile(file_name_rm)

findata_sheet = pd.read_excel(
    fin_file, sheet_name="main", index_col=0, header=0)

porfolio_selection = pd.DataFrame(findata_sheet)


porfolio_selection.head()


,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2017,2017-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,77.471831,NaN,NaN,NaN,-0.6023,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2018,2018-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,88.120879,NaN,NaN,NaN,-6.3187,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2019,2019-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,29.957895,NaN,NaN,NaN,-5.9765,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2020,2020-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,-134.845070,NaN,NaN,NaN,-33.5734,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2021,2021-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,-363.140845,NaN,NaN,NaN,-6.8196,NaN,NaN,NaN


## Sorting the data by firm name and then date

In [2731]:
portfolio_selection_sorted = porfolio_selection.sort_values(
    by=['firm_name', 'date'])
portfolio_selection_sorted.head(10)

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
4,AEM HOLDINGS LTD,AEM SP Equity,8,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-12.146377,7.806869,NaN,NaN,0.703200,4.012265,NaN,NaN
4,AEM HOLDINGS LTD,AEM SP Equity,8,2018,2018-12-31,41.625591,19.601330,24.607014,80.523781,482.330191,...,NaN,NaN,71.878555,221.110924,NaN,NaN,2.760650,1.450771,NaN,NaN
4,AEM HOLDINGS LTD,AEM SP Equity,8,2019,2019-12-31,NaN,NaN,NaN,NaN,NaN,...,-0.800330,30.824737,60.715333,95.638409,-0.312538,-0.090824,-0.351300,12.194501,NaN,NaN
4,AEM HOLDINGS LTD,AEM SP Equity,8,2020,2020-12-31,44.381851,34.913921,16.989117,81.095726,593.156683,...,-0.772134,9.850877,63.316760,70.446577,-0.758956,-0.538899,11.809200,13.248795,1.723032,21.921476
4,AEM HOLDINGS LTD,AEM SP Equity,8,2021,2021-12-31,44.804344,36.182423,16.989117,81.095726,614.707400,...,-0.838960,13.594900,-34.042491,113.952962,0.418044,-0.096504,7.532840,6.473027,1.552343,22.353920
4,AEM HOLDINGS LTD,AEM SP Equity,8,2022,2022-12-30,53.847702,43.823620,36.517532,81.095726,1600.330455,...,-0.752555,-18.978947,46.266254,101.846168,-0.640625,-0.018974,6.473700,7.180211,1.428175,26.914364
4,AEM HOLDINGS LTD,AEM SP Equity,8,2023,2023-01-31,NaN,NaN,NaN,NaN,NaN,...,-0.840080,-18.759538,45.768670,97.622491,-0.660997,-0.013433,6.473700,7.180211,1.428175,NaN
74,AIMS APAC REIT,AAREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN
74,AIMS APAC REIT,AAREIT SP Equity,11,2018,2018-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,20.764662,-18.426637,299.862490,0.130698,0.130698,4.767879,1.755459,NaN,NaN


## Ranking stocks percentile
| Country/Region | Percentile chosen (Top(%))|
| --- | --- |
| Australia (AU)| 20 |
| China (CN)| 20 |
| Malaysia (SG)| 20 |
| Hong Kong (SG)| 25 |
| Singapore (SG)| 25 |

## Excess Return
At the end of each month, compute the cross-sectional portfolio return of esg_top20. For example, at the end of August, after selecting the top 20% stocks based on August ESG scores, compute the raw returns of each stocks in the high-rated portfolio (the top 20%) by the following equation: $$ret_{i}  = P_{1}/P_{0} -1 $$(for example, after ranking stocks at the end of Aug, calculate the individual returns by using $$(Sept-price/Aug-price) – 1)$$ Next, compute excess returns of each stocks: $$r_{i} = ret_{i} – r_{f}$$ where r_i is the excess return of stock i and r_f is the respective country’s monthly risk-free rate. So now we have r_1, …, r_n where n = number of stocks in the high-rated portfolio. Next, we equal-weight (average) all the r in the esg_top 20 and find the portfolio mean (denoted as μ_t) for each month. This generates the High portfolio excess return (equal weight) in the Excel file for each month. Scroll down the Excel file, compute the mean of all the months’ excess returns, and test for significance (H0: mean > 0). Indicate the t-stat value.

In [2732]:
"""
change data format
"""

portfolio_selection_sorted['date'] = portfolio_selection_sorted['date'].astype(
    'datetime64')
portfolio_selection_sorted.dtypes


firm_name                        object
tkr                              object
sec_no                            int64
year                              int64
date                     datetime64[ns]
esg                             float64
e                               float64
s                               float64
g                               float64
es                              float64
eg                              float64
sg                              float64
esg_cubic                       float64
px                              float64
fcf                             float64
pe                              float64
beta                            float64
roa                             float64
vol_180d                        float64
sales                           float64
ret                             float64
historical_market_cap           float64
ep                              float64
ep_sec_mean                     float64
ep_sec_sd                       float64


In [2733]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
quantile_rank = 0
quantile_path = ''
quantile_folder = ''


quantile_folder = "..\..\Sray\Analysis_top\\" + file_name[:2] + "\\" + indicator
final_quantile_folder = "..\..\Sray\Analysis\\" + file_name[:2] + "\\" + indicator


# quantile_path = final_quantile_folder + "\\" + file_name[:2] +"_portfolio_" + indicator + "_quantile_rank.xlsx"
top_path = final_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_top20_second_highest.xlsx"
reporting_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_second_highest.xlsx"

rfr_sheet = pd.read_excel(rfr_file, sheet_name= file_name[:2] )
rm_sheet = pd.read_excel(rm_file, sheet_name= file_name[:2] )

try:
    os.makedirs(quantile_folder)
    os.makedirs(final_quantile_folder)
except FileExistsError:
    # directory already exists
    pass

if file_name[:2].lower() == 'AU'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'CN'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'HK'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'MY'.lower():
    quantile_rank = 4

elif file_name[:2].lower() == 'SG'.lower():
    quantile_rank = 4
    

In [2734]:
rfr_sheet = rfr_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rfr_yr': 'rfr'}, axis=1)
rfr_sheet

,year,rfr
0,2017,0.01762
1,2018,0.02013
2,2019,0.01657
3,2020,0.00305
4,2021,0.00442


In [2735]:
rm_sheet = rm_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rm_yr': 'rm'}, axis=1)
rm_sheet

,year,rm
0,2017,0.116881
1,2018,-0.131644
2,2019,0.010238
3,2020,-0.098272
4,2021,0.108782


In [2736]:
rfr_rm_sheet = pd.merge(rfr_sheet, rm_sheet, how='left', on='year')
rfr_rm_sheet

,year,rfr,rm
0,2017,0.01762,0.116881
1,2018,0.02013,-0.131644
2,2019,0.01657,0.010238
3,2020,0.00305,-0.098272
4,2021,0.00442,0.108782


In [2737]:
"""
gets all the companies, dropping the empty columns
"""
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates()

#set the data to a list
companies = companies.tolist()


#ensure the number of companies exist
len(companies)


105

In [2738]:
# new_companies_data.to_excel("SG_data_2.28_esgj3mean.xlsx")

### Rank the companies 

In [2739]:
companies_sray = []
all_quantile_rank = []

initial_year = 2017
stop_year = 2023


for year in range(initial_year, stop_year+1, 1):

        # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
        temp_dataframe = portfolio_selection_sorted.loc[(
            portfolio_selection_sorted['year'] == year)].copy()

        
        """
        sort by the relative value and then divide them to relative division
        """

        temp_dataframe.sort_values(by=[indicator], inplace=True)

        temp_dataframe['quantile_rank'] = pd.qcut(temp_dataframe[indicator],
                                                    q=quantile_rank, labels=False, duplicates='drop')
        

        temp_dataframe_quantile = temp_dataframe.copy()

        companies_sray.append(temp_dataframe)

        all_quantile_rank.append(temp_dataframe_quantile)


In [2740]:
all_sort_companies_data = pd.concat(all_quantile_rank)

# all_sort_companies_data.to_excel(
#     quantile_path, sheet_name="main")


In [2741]:
sort_companies_data = pd.concat(companies_sray)
sort_companies_data.head()

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
firm no,,,,,,,,,,,,,,,,,,,,,
98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,0.000000,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,1936.849257,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,205.008084,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


In [2742]:
all_sort_companies_data = all_sort_companies_data.reset_index()
all_sort_companies_data = pd.merge(all_sort_companies_data, rfr_rm_sheet, how='left', on='year')
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('ret')+1, 'rfr', all_sort_companies_data.pop('rfr'))
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('rfr')+1, 'rm', all_sort_companies_data.pop('rm'))

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


In [2743]:
df_sec_company_count_sum = []
df_sec_company_count = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].count())
df_sec_company_count = df_sec_company_count.rename({'tkr':'tkr_count'}, axis=1)
df_sec_company_nunique = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].nunique())
df_sec_company_nunique = df_sec_company_nunique.rename({'tkr':'tkr_nunique'}, axis=1)

df_sec_company_count = pd.merge(df_sec_company_count, df_sec_company_nunique, on=['quantile_rank','year'])

In [2744]:
df_sec_company_count_sum = list()
# for rank in range (0, quantile_rank, 1):
groups = df_sec_company_count.groupby(level='quantile_rank')

    # loop through the groups to create separate dataframes
dfs = {}
for name, group in groups:
    dfs[name] = group.copy()
        
    # display the separate dataframes
for name, df in dfs.items():

        # Create a new row with the sum of all values above it
    sum_row = df.sum()
    sum_index = (name, 'Sum')
    sum_row.name = sum_index

        # Append the new row to the original DataFrame
    temp_1 = df.append(sum_row)
    df_sec_company_count_sum.append(temp_1)
# df_sec_company_count_all = pd.concat(df_sec_company_count_sum)
df_sec_company_count_sum_pd = pd.concat(df_sec_company_count_sum)

In [2745]:
# book = load_workbook(quantile_path)
# writer = pd.ExcelWriter(quantile_path, engine = 'openpyxl')
# writer.book = book
# df_sec_company_count_sum_pd.to_excel(writer, sheet_name='quantile sec no', index=True)
# writer.close()

In [2746]:
# sort_companies_data['date'] = sort_companies_data['date'].dt.strftime(
#     '%Y-%m')

all_sort_companies_data = all_sort_companies_data.sort_values(
    by=["date", "quantile_rank"])

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


**SRAY- Ret already calculated using excel by Dr May**
<br>**BLOOMBERG- Ret calculated using PYTHON CODE 2_portfolio.ipynb**

In [2747]:

#set the query string
query_str = ''
query_str = 'quantile_rank == ' + str(quantile_rank-2)

sort_companies_data_top20per = all_sort_companies_data.query(query_str)

sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
47,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,2.0
48,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,13.856335,41.664941,101.757100,-0.273284,-0.456677,4.371840,4.429012,1.564584,20.629724,2.0
49,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,13.648881,41.664941,101.757100,-0.275323,-0.252285,4.371840,4.429012,-0.209288,13.479129,2.0
50,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,2.0
51,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,2.0


In [2748]:
sort_companies_data_top20per.columns

Index(['firm no', 'firm_name', 'tkr', 'sec_no', 'year', 'date', 'esg', 'e',
       's', 'g', 'es', 'eg', 'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta',
       'roa', 'vol_180d', 'sales', 'ret', 'rfr', 'rm', 'historical_market_cap',
       'ep', 'ep_sec_mean', 'ep_sec_sd', 'ep_z', 'sp', 'sp_sec_mean',
       'sp_sec_sd', 'sp_z', 'cfp', 'cfp_sec_mean', 'cfp_sec_sd', 'cfp_z',
       'value', 'roa_sec_mean', 'roa_sec_sd', 'roa_z', 'esg_value',
       'quantile_rank'],
      dtype='object')

In [2749]:
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates().tolist()
print(companies)

['AEM HOLDINGS LTD', 'AIMS APAC REIT', 'APAC REALTY LTD', 'ARA US HOSPITALITY TRUST', 'ASPEN GROUP HOLDINGS LTD', 'BANYAN TREE HOLDINGS LTD', 'BUKIT SEMBAWANG ESTATES LTD', 'BUMITAMA AGRI LTD', 'BUND CENTER INVESTMENT LTD', 'CAPITALAND ASCENDAS REIT', 'CAPITALAND ASCOTT TRUST', 'CAPITALAND CHINA TRUST', 'CAPITALAND INDIA TRUST', 'CAPITALAND INTEGRATED COMMER', 'CAPITALAND INVESTMENT LTD/SI', 'CDL HOSPITALITY TRUSTS', 'CENTURION CORP LTD', 'CHINA FISHERY GROUP LTD', 'CHINA SUNSINE CHEMICAL HLDGS', 'CHIP ENG SENG CORP LTD', 'CITY DEVELOPMENTS LTD', 'COMFORTDELGRO CORP LTD', 'COSCO SHIPPING INTERNATIONAL', 'CROMWELL REIT EUR', 'DBS GROUP HOLDINGS LTD', 'DEL MONTE PACIFIC LTD', 'DELFI LTD', 'DFI RETAIL GROUP HOLDINGS LT', 'EC WORLD REIT', 'ESR-LOGOS REIT', 'EZRA HOLDINGS LTD', 'FAR EAST HOSPITALITY TRUST', 'FIRST RESOURCES LTD', 'FIRST SPONSOR GROUP LTD', 'FRASER AND NEAVE LTD', 'FRASERS CENTREPOINT TRUST', 'FRASERS LOGISTICS & COMMERCI', 'FRASERS PROPERTY LTD', 'GENTING SINGAPORE LTD', 'G

In [2750]:
companies_sec=[]
initial_year=2017
stop_year=2023

company_dict1 = {}
company_dict_all = {}
sum_esg_sray = {}
sum_ret_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =0 and save it into company dictionary 
for year in range(initial_year,stop_year+1,1):    

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        each_year_companies_all_quantile = all_sort_companies_data.loc[(
            all_sort_companies_data['year'] == year)]

        sum = each_year_companies[indicator].sum()
        sum_esg_sray.setdefault(year, sum)

        sum_ret = each_year_companies['ret'].sum()
        sum_ret_dict.setdefault(year, sum_ret)

        companies = each_year_companies.iloc[:, 0]
        companies= companies.tolist() 

        total_companies = (each_year_companies_all_quantile.iloc[:, 0]).tolist()


        for c in range (len(companies)):
            company_dict1.setdefault(year,[]).append(companies[c])
            #company_dict2.setdefault(start_date,[]).append(companies[c])
        for c_all in range(len(total_companies)):
            company_dict_all.setdefault(
                year, []).append(total_companies[c_all])    
        

#make sure the list is correct     
#print(company_dict)
# print(f"company_dict1: {company_dict1}")
print(len(company_dict1))
print(f"sum_esg_sray_dic: {sum_esg_sray}")
print(f"sum_ret_dict: {sum_ret_dict}")
# print(len(company_dict2))


7
sum_esg_sray_dic: {2017: 500.68100000000004, 2018: 675.557, 2019: 595.388, 2020: 813.196, 2021: 590.541, 2022: 706.178, 2023: 683.826}
sum_ret_dict: {2017: 0.0, 2018: -4.498313636190602, 2019: 2.240484836748539, 2020: -4.890257083220937, 2021: 1.695265298837768, 2022: -0.6750434549141137, 2023: 1.607894365538308}


If it is equal-weighted, compute the mean of the stocks returns in the portfolio. 
$μ_(p,t)=  (\sum ret_{(i,t)} )/n$; i = 1, 2, 3, …, n
Where $ret_(i,t) = return-of-stock-i-at-time-t$, n = number of stocks in the portfolio


In [2751]:
no_of_companies = {}
equal_weight_dict = {}
for date_range, company_names in company_dict1.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    #equal weightage
    # equal_weight = np.repeat(1/len(v), len(v))
    equal_weight = 1/len(company_names)
        # print(f"factor_weight: {factor_weight}")

    equal_weight_dict.setdefault(date_range, float(equal_weight))
    no_of_companies.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"equal_weight: {equal_weight_dict}")
print(f"no_of_companies: {no_of_companies}")


equal_weight: {2017: 0.043478260869565216, 2018: 0.041666666666666664, 2019: 0.04, 2020: 0.04, 2021: 0.04, 2022: 0.04, 2023: 0.04}
no_of_companies: {2017: 23.0, 2018: 24.0, 2019: 25.0, 2020: 25.0, 2021: 25.0, 2022: 25.0, 2023: 25.0}


### total companies from all quantile

In [2752]:
no_of_companies_all = {}
for date_range, company_names in company_dict_all.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    no_of_companies_all.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"no_of_companies: {no_of_companies_all}")


no_of_companies: {2017: 105.0, 2018: 105.0, 2019: 105.0, 2020: 105.0, 2021: 105.0, 2022: 105.0, 2023: 105.0}


## Calculate the number of companies (top 20%) based on the date

In [2753]:
no_of_companies_items = no_of_companies.items()
no_of_companies_list = list(no_of_companies_items)

no_of_companies_df = pd.DataFrame(no_of_companies_list)
no_of_companies_df.columns = ['year', 'no of firms(top 20)']
no_of_companies_df.head()

,year,no of firms(top 20)
0,2017,23.0
1,2018,24.0
2,2019,25.0
3,2020,25.0
4,2021,25.0


## Calculate the number of companies based on the date

In [2754]:
no_of_companies_all_items = no_of_companies_all.items()
no_of_companies_all_list = list(no_of_companies_all_items)

no_of_companies_all_df = pd.DataFrame(no_of_companies_all_list)
no_of_companies_all_df.columns = ['year', 'no of firms']
no_of_companies_all_df.head()

,year,no of firms
0,2017,105.0
1,2018,105.0
2,2019,105.0
3,2020,105.0
4,2021,105.0


### Make the equal weight item into dataframe
Purpose: use for merging later on

In [2755]:
equal_weight_items = equal_weight_dict.items()
equal_weight_list = list(equal_weight_items)

equal_weight_df = pd.DataFrame(equal_weight_list)
equal_weight_df.columns = ['year', 'eq_wg']
equal_weight_df.head()


,year,eq_wg
0,2017,0.043478
1,2018,0.041667
2,2019,0.040000
3,2020,0.040000
4,2021,0.040000


In [2756]:
# no_of_companies_df.to_excel("eq_wg_no_of_companies_top20.xlsx")

In [2757]:
sum_ret_items = sum_ret_dict.items()
sum_ret_list = list(sum_ret_items)

sum_ret_df = pd.DataFrame(sum_ret_list)
sum_ret_df.columns = ['year', 'sum_ret']
sum_ret_df.replace(0, np.nan, inplace=True)
sum_ret_df.head()


,year,sum_ret
0,2017,NaN
1,2018,-4.498314
2,2019,2.240485
3,2020,-4.890257
4,2021,1.695265


### Make the sum of the esg_sray into dataframe
Purpose: merge into complete dataframe to make the data to be able to vefiry

In [2758]:
sum_esg_sray_dic_items = sum_esg_sray.items()
sum_esg_sray_list = list(sum_esg_sray_dic_items)

sum_esg_sray_df = pd.DataFrame(sum_esg_sray_list)
sum_esg_sray_df.columns = ['year', 'sum_'+indicator]
sum_esg_sray_df.head()


,year,sum_vol_180d
0,2017,500.681
1,2018,675.557
2,2019,595.388
3,2020,813.196
4,2021,590.541


### Merging with the data of top 20% **only**

In [2759]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    equal_weight_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    sum_esg_sray_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    sum_ret_df, on='year')
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,NaN,5.630938,3.148304,NaN,NaN,2.0,0.043478,500.681,NaN
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,-0.273284,-0.456677,4.371840,4.429012,1.564584,20.629724,2.0,0.043478,500.681,NaN
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,-0.275323,-0.252285,4.371840,4.429012,-0.209288,13.479129,2.0,0.043478,500.681,NaN
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,NaN,5.630938,3.148304,NaN,NaN,2.0,0.043478,500.681,NaN
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,NaN,5.630938,3.148304,NaN,NaN,2.0,0.043478,500.681,NaN


### Calculate the factor weight

In [2760]:
if sort_companies_data_top20per[indicator].empty and sort_companies_data_top20per['sum_'+indicator].empty:
    sort_companies_data_top20per['fc_wg'] = None
else:
    sort_companies_data_top20per['fc_wg'] = sort_companies_data_top20per[indicator] / \
        sort_companies_data_top20per['sum_'+indicator]

sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,5.630938,3.148304,NaN,NaN,2.0,0.043478,500.681,NaN,0.036906
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,-0.456677,4.371840,4.429012,1.564584,20.629724,2.0,0.043478,500.681,NaN,0.037347
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,-0.252285,4.371840,4.429012,-0.209288,13.479129,2.0,0.043478,500.681,NaN,0.037449
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,5.630938,3.148304,NaN,NaN,2.0,0.043478,500.681,NaN,0.037818
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,5.630938,3.148304,NaN,NaN,2.0,0.043478,500.681,NaN,0.037850


In [2761]:
sort_companies_data_top20per['r'] = (
    sort_companies_data_top20per['ret'] - sort_companies_data_top20per['rfr'])

"""
r = ret -rfr
"""
sort_companies_data_top20per['eq_wg_r'] = (
    sort_companies_data_top20per['r'] * sort_companies_data_top20per['eq_wg'])
sort_companies_data_top20per['fc_wg_r'] = (
    sort_companies_data_top20per['r'] * sort_companies_data_top20per['fc_wg'])

"""
ret
"""
sort_companies_data_top20per['eq_wg_ret'] = (
    sort_companies_data_top20per['ret'] * sort_companies_data_top20per['eq_wg'])
sort_companies_data_top20per['fc_wg_ret'] = (
    sort_companies_data_top20per['ret'] * sort_companies_data_top20per['fc_wg'])

sort_companies_data_top20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,2.0,0.043478,500.681,NaN,0.036906,NaN,NaN,NaN,NaN,NaN
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,2.0,0.043478,500.681,NaN,0.037347,NaN,NaN,NaN,NaN,NaN
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,2.0,0.043478,500.681,NaN,0.037449,NaN,NaN,NaN,NaN,NaN
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,2.0,0.043478,500.681,NaN,0.037818,NaN,NaN,NaN,NaN,NaN
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,2.0,0.043478,500.681,NaN,0.037850,NaN,NaN,NaN,NaN,NaN
5,106,OLAM GROUP LTD,OLG SP Equity,5,2017,2017-12-31,40.961674,26.759287,14.873035,81.095726,...,2.0,0.043478,500.681,NaN,0.037864,NaN,NaN,NaN,NaN,NaN
6,76,DBS GROUP HOLDINGS LTD,DBS SP Equity,7,2017,2017-12-31,42.480637,22.470552,18.712214,86.092712,...,2.0,0.043478,500.681,NaN,0.040762,NaN,NaN,NaN,NaN,NaN
7,7,RAFFLES MEDICAL GROUP LTD,RFMD SP Equity,6,2017,2017-12-31,37.350368,5.527031,25.272068,81.095726,...,2.0,0.043478,500.681,NaN,0.041414,NaN,NaN,NaN,NaN,NaN
8,35,CAPITALAND INDIA TRUST,CLINT SP Equity,11,2017,2017-12-31,37.621971,31.803082,17.382105,63.576160,...,2.0,0.043478,500.681,NaN,0.041541,NaN,NaN,NaN,NaN,NaN
9,24,LIPPO MALLS INDONESIA RETAIL,LMRT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,2.0,0.043478,500.681,NaN,0.041737,NaN,NaN,NaN,NaN,NaN


## Excess return portfolio
### Equal Weight and Factor Weight

In [2762]:
initial_year = 2017
stop_year = 2023

eq_wg_r_p_dict = {}
fc_wg_r_p_dict = {}
eq_wg_ret_p_dict = {}
fc_wg_ret_p_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        """
        r
        """
        eq_wg_r_p = each_year_companies['eq_wg_r'].sum()
        fc_wg_r_p = each_year_companies['fc_wg_r'].sum()


        """
        ret
        """
        eq_wg_ret_p = each_year_companies['eq_wg_ret'].sum()
        fc_wg_ret_p = each_year_companies['fc_wg_ret'].sum()


        eq_wg_r_p_dict.setdefault(year, eq_wg_r_p)
        fc_wg_r_p_dict.setdefault(year, fc_wg_r_p)

        eq_wg_ret_p_dict.setdefault(year, eq_wg_ret_p)
        fc_wg_ret_p_dict.setdefault(year, fc_wg_ret_p)


print(f"eq_wg_r_p_dict: {eq_wg_r_p_dict}")
print(f"fc_wg_r_p_dict: {fc_wg_r_p_dict}")
print(f"eq_wg_ret_p_dict: {eq_wg_ret_p_dict}")
print(f"fc_wg_ret_p_dict: {fc_wg_ret_p_dict}")
# print(len(company_dict2))


eq_wg_r_p_dict: {2017: 0.0, 2018: -0.20672098484127507, 2019: 0.07304939346994155, 2020: -0.19866028332883745, 2021: 0.06356741195351072, 2022: 0.0, 2023: 0.0}
fc_wg_r_p_dict: {2017: 0.0, 2018: -0.21557526510264235, 2019: 0.0691669640431653, 2020: -0.1950136638710509, 2021: 0.06211929701898881, 2022: 0.0, 2023: 0.0}
eq_wg_ret_p_dict: {2017: 0.0, 2018: -0.1874297348412751, 2019: 0.08961939346994155, 2020: -0.19561028332883745, 2021: 0.06781061195351071, 2022: -0.027001738196564557, 2023: 0.06431577462153232}
fc_wg_ret_p_dict: {2017: 0.0, 2018: -0.1962185136960253, 2019: 0.08573696404316529, 2020: -0.19196366387105088, 2021: 0.0663378323620048, 2022: -0.02611862002279419, 2023: 0.06443530483638625}


In [2763]:
eq_wg_r_p_dict_items = eq_wg_r_p_dict.items()
eq_wg_r_p_dict_list = list(eq_wg_r_p_dict_items)

eq_wg_r_p_dict_df = pd.DataFrame(eq_wg_r_p_dict_list)
eq_wg_r_p_dict_df.columns = ['year', 'eq_wg_r_p']
eq_wg_r_p_dict_df.replace(0, np.nan, inplace = True)
eq_wg_r_p_dict_df.head()

,year,eq_wg_r_p
0,2017,NaN
1,2018,-0.206721
2,2019,0.073049
3,2020,-0.198660
4,2021,0.063567


In [2764]:
fc_wg_r_p_dict_items = fc_wg_r_p_dict.items()
fc_wg_r_p_dict_list = list(fc_wg_r_p_dict_items)

fc_wg_r_p_dict_df = pd.DataFrame(fc_wg_r_p_dict_list)
fc_wg_r_p_dict_df.columns = ['year', 'fc_wg_r_p']
fc_wg_r_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_r_p_dict_df.head()


,year,fc_wg_r_p
0,2017,NaN
1,2018,-0.215575
2,2019,0.069167
3,2020,-0.195014
4,2021,0.062119


In [2765]:
eq_wg_ret_p_dict_items = eq_wg_ret_p_dict.items()
eq_wg_ret_p_dict_list = list(eq_wg_ret_p_dict_items)

eq_wg_ret_p_dict_df = pd.DataFrame(eq_wg_ret_p_dict_list)
eq_wg_ret_p_dict_df.columns = ['year', 'eq_wg_ret_p']
eq_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_ret_p_dict_df.head()


,year,eq_wg_ret_p
0,2017,NaN
1,2018,-0.187430
2,2019,0.089619
3,2020,-0.195610
4,2021,0.067811


In [2766]:
fc_wg_ret_p_dict_items = fc_wg_ret_p_dict.items()
fc_wg_ret_p_dict_list = list(fc_wg_ret_p_dict_items)

fc_wg_ret_p_dict_df = pd.DataFrame(fc_wg_ret_p_dict_list)
fc_wg_ret_p_dict_df.columns = ['year', 'fc_wg_ret_p']
fc_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_ret_p_dict_df.head()


,year,fc_wg_ret_p
0,2017,NaN
1,2018,-0.196219
2,2019,0.085737
3,2020,-0.191964
4,2021,0.066338


In [2767]:
SG_esg_int = pd.merge(no_of_companies_all_df, no_of_companies_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,2017,105.0,23.0,NaN,NaN,NaN,NaN
1,2018,105.0,24.0,-0.206721,-0.215575,-0.187430,-0.196219
2,2019,105.0,25.0,0.073049,0.069167,0.089619,0.085737
3,2020,105.0,25.0,-0.198660,-0.195014,-0.195610,-0.191964
4,2021,105.0,25.0,0.063567,0.062119,0.067811,0.066338


### Merging the eq and fc wg of the r_p into the big dataframe

In [2768]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_r_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_r_p_dict_df, on='year')

sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_ret_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_ret_p_dict_df, on='year')

sort_companies_data_top20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret,eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,0.036906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,0.037347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,0.037449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,0.037818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,0.037850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,106,OLAM GROUP LTD,OLG SP Equity,5,2017,2017-12-31,40.961674,26.759287,14.873035,81.095726,...,0.037864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,76,DBS GROUP HOLDINGS LTD,DBS SP Equity,7,2017,2017-12-31,42.480637,22.470552,18.712214,86.092712,...,0.040762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,RAFFLES MEDICAL GROUP LTD,RFMD SP Equity,6,2017,2017-12-31,37.350368,5.527031,25.272068,81.095726,...,0.041414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,35,CAPITALAND INDIA TRUST,CLINT SP Equity,11,2017,2017-12-31,37.621971,31.803082,17.382105,63.576160,...,0.041541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,24,LIPPO MALLS INDONESIA RETAIL,LMRT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.041737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2769]:
sort_companies_data_top20per['rm_r_y1'] = (
    sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr']) * 12
sort_companies_data_top20per['rm_r_y2'] = (((1+(
    sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr'])) ** 12) - 1)

sort_companies_data_top20per['eq_wg_r_p_y1'] = sort_companies_data_top20per['eq_wg_r_p'] *12
sort_companies_data_top20per['eq_wg_r_p_y2'] = (((1 + sort_companies_data_top20per['eq_wg_r_p'])**12) - 1)
sort_companies_data_top20per['eq_r_alpha_y1'] = sort_companies_data_top20per['eq_wg_r_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['eq_r_alpha_y2'] = sort_companies_data_top20per['eq_wg_r_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN


In [2770]:
sort_companies_data_top20per['fc_wg_r_p_y1'] = sort_companies_data_top20per['fc_wg_r_p'] * 12
sort_companies_data_top20per['fc_wg_r_p_y2'] = ((
    (1 + sort_companies_data_top20per['fc_wg_r_p'])**12) - 1)
sort_companies_data_top20per['fc_r_alpha_y1'] = sort_companies_data_top20per['fc_wg_r_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['fc_r_alpha_y2'] = sort_companies_data_top20per['fc_wg_r_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2771]:

# sort_companies_data_top20per.to_excel(
#     "../3.04/SG_esg_top20.xlsx", sheet_name="main")

## CAPM-adjusted return ($r_{c}$)

$r_(c,t)=r_(f,t)+β_(p,t) (r_(m,t)-r_(f,t))$

In [2772]:
sort_companies_data_top20per['rc'] = sort_companies_data_top20per['rfr'] + \
    (sort_companies_data_top20per['beta'] *
    (sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr']))
    
"""
equal weight rc
"""
sort_companies_data_top20per['eq_wg_rc_int'] = sort_companies_data_top20per ['eq_wg'] * sort_companies_data_top20per['rc']

"""
factor weight rc
"""
sort_companies_data_top20per['fc_wg_rc_int'] = sort_companies_data_top20per['fc_wg'] * \
    sort_companies_data_top20per['rc']

sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127283,0.005534,0.004697
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.074467,0.003238,0.002781
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003701,-0.000161,-0.000139
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039418,0.001714,0.001491
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.139979,0.006086,0.005298


In [2773]:
initial_year = 2017
stop_year = 2023

eq_wg_rc_p_dict = {}
fc_wg_rc_p_dict = {}

# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        """
        rc
        """
        eq_wg_rc_p = each_year_companies['eq_wg_rc_int'].sum()
        fc_wg_rc_p = each_year_companies['fc_wg_rc_int'].sum()

        eq_wg_rc_p_dict.setdefault(year, eq_wg_rc_p)
        fc_wg_rc_p_dict.setdefault(year, fc_wg_rc_p)


print(f"eq_wg_rc_p_dict: {eq_wg_rc_p_dict}")
print(f"fc_wg_rc_p_dict: {fc_wg_rc_p_dict}")


eq_wg_rc_p_dict: {2017: 0.10243131796983268, 2018: -0.12476369184024114, 2019: 0.01184754257340712, 2020: -0.09815774626232622, 2021: 0.12435356792154387, 2022: 0.0, 2023: 0.0}
fc_wg_rc_p_dict: {2017: 0.10249600004234413, 2018: -0.12907431757315538, 2019: 0.01184499590428592, 2020: -0.09907568530058462, 2021: 0.1239208190495887, 2022: 0.0, 2023: 0.0}


In [2774]:
eq_wg_rc_p_dict_items = eq_wg_rc_p_dict.items()
eq_wg_rc_p_dict_list = list(eq_wg_rc_p_dict_items)

eq_wg_rc_p_dict_df = pd.DataFrame(eq_wg_rc_p_dict_list)
eq_wg_rc_p_dict_df.columns = ['year', 'eq_wg_rc_p']
eq_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_rc_p_dict_df.head()


,year,eq_wg_rc_p
0,2017,0.102431
1,2018,-0.124764
2,2019,0.011848
3,2020,-0.098158
4,2021,0.124354


In [2775]:
fc_wg_rc_p_dict_items = fc_wg_rc_p_dict.items()
fc_wg_rc_p_dict_list = list(fc_wg_rc_p_dict_items)

fc_wg_rc_p_dict_df = pd.DataFrame(fc_wg_rc_p_dict_list).dropna()
fc_wg_rc_p_dict_df.columns = ['year', 'fc_wg_rc_p']
fc_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_rc_p_dict_df.head()


,year,fc_wg_rc_p
0,2017,0.102496
1,2018,-0.129074
2,2019,0.011845
3,2020,-0.099076
4,2021,0.123921


In [2776]:
SG_esg_int = pd.merge(SG_esg_int, eq_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()

,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,eq_wg_rc_p,fc_wg_rc_p
0,2017,105.0,23.0,NaN,NaN,NaN,NaN,0.102431,0.102496
1,2018,105.0,24.0,-0.206721,-0.215575,-0.187430,-0.196219,-0.124764,-0.129074
2,2019,105.0,25.0,0.073049,0.069167,0.089619,0.085737,0.011848,0.011845
3,2020,105.0,25.0,-0.198660,-0.195014,-0.195610,-0.191964,-0.098158,-0.099076
4,2021,105.0,25.0,0.063567,0.062119,0.067811,0.066338,0.124354,0.123921


In [2777]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_rc_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_rc_p_dict_df, on='year')

sort_companies_data_top20per.head(15)

,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int,eq_wg_rc_p,fc_wg_rc_p
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,NaN,NaN,NaN,NaN,0.127283,0.005534,0.004697,0.102431,0.102496
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,NaN,NaN,NaN,NaN,NaN,0.074467,0.003238,0.002781,0.102431,0.102496
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,NaN,NaN,NaN,NaN,NaN,-0.003701,-0.000161,-0.000139,0.102431,0.102496
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,NaN,NaN,NaN,NaN,0.039418,0.001714,0.001491,0.102431,0.102496
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,NaN,NaN,NaN,NaN,0.139979,0.006086,0.005298,0.102431,0.102496
5,106,OLAM GROUP LTD,OLG SP Equity,5,2017,2017-12-31,40.961674,26.759287,14.873035,81.095726,...,NaN,NaN,NaN,NaN,NaN,0.084512,0.003674,0.003200,0.102431,0.102496
6,76,DBS GROUP HOLDINGS LTD,DBS SP Equity,7,2017,2017-12-31,42.480637,22.470552,18.712214,86.092712,...,NaN,NaN,NaN,NaN,NaN,0.177489,0.007717,0.007235,0.102431,0.102496
7,7,RAFFLES MEDICAL GROUP LTD,RFMD SP Equity,6,2017,2017-12-31,37.350368,5.527031,25.272068,81.095726,...,NaN,NaN,NaN,NaN,NaN,0.045532,0.001980,0.001886,0.102431,0.102496
8,35,CAPITALAND INDIA TRUST,CLINT SP Equity,11,2017,2017-12-31,37.621971,31.803082,17.382105,63.576160,...,NaN,NaN,NaN,NaN,NaN,0.078457,0.003411,0.003259,0.102431,0.102496
9,24,LIPPO MALLS INDONESIA RETAIL,LMRT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.068233,0.002967,0.002848,0.102431,0.102496


In [2778]:
sort_companies_data_top20per['eq_wg_rc_p_y1'] = sort_companies_data_top20per['eq_wg_rc_p'] * 12
sort_companies_data_top20per['eq_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_top20per['eq_wg_rc_p'])**12) - 1)
sort_companies_data_top20per['eq_rc_alpha_y1'] = sort_companies_data_top20per['eq_wg_rc_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['eq_rc_alpha_y2'] = sort_companies_data_top20per['eq_wg_rc_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int,eq_wg_rc_p,fc_wg_rc_p,eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,NaN,0.127283,0.005534,0.004697,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,NaN,0.074467,0.003238,0.002781,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,NaN,-0.003701,-0.000161,-0.000139,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,NaN,0.039418,0.001714,0.001491,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,NaN,0.139979,0.006086,0.005298,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482


In [2779]:
sort_companies_data_top20per['fc_wg_rc_p_y1'] = sort_companies_data_top20per['fc_wg_rc_p'] * 12
sort_companies_data_top20per['fc_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_top20per['fc_wg_rc_p'])**12) - 1)
sort_companies_data_top20per['fc_rc_alpha_y1'] = sort_companies_data_top20per['fc_wg_rc_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['fc_rc_alpha_y2'] = sort_companies_data_top20per['fc_wg_rc_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_wg_rc_p,fc_wg_rc_p,eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2,fc_wg_rc_p_y1,fc_wg_rc_p_y2,fc_rc_alpha_y1,fc_rc_alpha_y2
0,48,UOL GROUP LTD,UOL SP Equity,11,2017,2017-12-31,41.615532,20.477198,23.125755,81.095726,...,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482,1.229952,2.22496,0.038824,0.111751
1,29,SATS LTD,SATS SP Equity,3,2017,2017-12-31,41.716125,26.970703,19.437727,78.597229,...,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482,1.229952,2.22496,0.038824,0.111751
2,13,JARDINE CYCLE & CARRIAGE LTD,JCNC SP Equity,3,2017,2017-12-31,27.210543,0.000000,5.350665,76.098740,...,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482,1.229952,2.22496,0.038824,0.111751
3,15,GUOCOLAND LTD,GUOL SP Equity,11,2017,2017-12-31,28.729504,0.000000,4.897219,81.095726,...,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482,1.229952,2.22496,0.038824,0.111751
4,18,WING TAI HOLDINGS LTD,WINGT SP Equity,11,2017,2017-12-31,25.641285,0.000000,0.634825,76.098740,...,0.102431,0.102496,1.229176,2.22269,0.038048,0.109482,1.229952,2.22496,0.038824,0.111751


In [2780]:
SG_int_wanted = sort_companies_data_top20per[[
    'year', 'rm_r_y1', 'rm_r_y2', 'eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_r_alpha_y2', 'eq_wg_r_p_y2', 'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_r_alpha_y2', 'fc_wg_r_p_y2', 'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_rc_alpha_y2', 'eq_wg_rc_p_y2', 'fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_rc_alpha_y2', 'fc_wg_rc_p_y2']]
SG_int_wanted = SG_int_wanted.drop_duplicates()
SG_int_wanted.head()


,year,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_r_alpha_y1,eq_r_alpha_y2,eq_wg_r_p_y2,fc_wg_r_p_y1,fc_r_alpha_y1,fc_r_alpha_y2,fc_wg_r_p_y2,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,fc_wg_rc_p_y2
0,2017,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.229176,0.038048,0.109482,2.222690,1.229952,0.038824,0.111751,2.224960
23,2018,-1.821289,-0.861280,-2.480652,-0.659362,-0.076617,-0.937897,-2.586903,-0.765614,-0.084443,-0.945723,-1.497164,0.324125,0.063351,-0.797929,-1.548892,0.272398,0.051727,-0.809553
47,2019,-0.075984,-0.073393,0.876593,0.952577,1.403826,1.330433,0.830004,0.905988,1.304634,1.231240,0.142171,0.218155,0.225204,0.151810,0.142140,0.218124,0.225169,0.151776
72,2020,-1.215864,-0.722509,-2.383923,-1.168059,-0.207378,-0.929887,-2.340164,-1.124300,-0.203452,-0.925961,-1.177893,0.037971,0.011954,-0.710554,-1.188908,0.026956,0.008439,-0.714070
97,2021,1.252343,2.291070,0.762809,-0.489534,-1.196088,1.094982,0.745432,-0.506912,-1.230063,1.061008,1.492243,0.239900,0.790571,3.081641,1.487050,0.234707,0.771759,3.062829


In [2781]:
SG_esg_int = pd.merge(SG_esg_int, SG_int_wanted,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,eq_wg_rc_p,fc_wg_rc_p,rm_r_y1,...,fc_r_alpha_y2,fc_wg_r_p_y2,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,fc_wg_rc_p_y2
0,2017,105.0,23.0,NaN,NaN,NaN,NaN,0.102431,0.102496,1.191128,...,NaN,NaN,1.229176,0.038048,0.109482,2.222690,1.229952,0.038824,0.111751,2.224960
1,2018,105.0,24.0,-0.206721,-0.215575,-0.187430,-0.196219,-0.124764,-0.129074,-1.821289,...,-0.084443,-0.945723,-1.497164,0.324125,0.063351,-0.797929,-1.548892,0.272398,0.051727,-0.809553
2,2019,105.0,25.0,0.073049,0.069167,0.089619,0.085737,0.011848,0.011845,-0.075984,...,1.304634,1.231240,0.142171,0.218155,0.225204,0.151810,0.142140,0.218124,0.225169,0.151776
3,2020,105.0,25.0,-0.198660,-0.195014,-0.195610,-0.191964,-0.098158,-0.099076,-1.215864,...,-0.203452,-0.925961,-1.177893,0.037971,0.011954,-0.710554,-1.188908,0.026956,0.008439,-0.714070
4,2021,105.0,25.0,0.063567,0.062119,0.067811,0.066338,0.124354,0.123921,1.252343,...,-1.230063,1.061008,1.492243,0.239900,0.790571,3.081641,1.487050,0.234707,0.771759,3.062829


In [2782]:
SG_esg_int = SG_esg_int[['year', 'no of firms', 'no of firms(top 20)',
                            'rm_r_y1', 'rm_r_y2', 'eq_wg_ret_p', 'fc_wg_ret_p', 'eq_wg_r_p',
                            'eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_wg_r_p_y2', 'eq_r_alpha_y2', 'fc_wg_r_p',
                            'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_wg_r_p_y2', 'fc_r_alpha_y2', 'eq_wg_rc_p',
                            'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2', 'eq_rc_alpha_y2', 'fc_wg_rc_p',
                            'fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_wg_rc_p_y2', 'fc_rc_alpha_y2']]
SG_esg_int.head()


,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
0,2017,105.0,23.0,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,...,0.102431,1.229176,0.038048,2.222690,0.109482,0.102496,1.229952,0.038824,2.224960,0.111751
1,2018,105.0,24.0,-1.821289,-0.861280,-0.187430,-0.196219,-0.206721,-2.480652,-0.659362,...,-0.124764,-1.497164,0.324125,-0.797929,0.063351,-0.129074,-1.548892,0.272398,-0.809553,0.051727
2,2019,105.0,25.0,-0.075984,-0.073393,0.089619,0.085737,0.073049,0.876593,0.952577,...,0.011848,0.142171,0.218155,0.151810,0.225204,0.011845,0.142140,0.218124,0.151776,0.225169
3,2020,105.0,25.0,-1.215864,-0.722509,-0.195610,-0.191964,-0.198660,-2.383923,-1.168059,...,-0.098158,-1.177893,0.037971,-0.710554,0.011954,-0.099076,-1.188908,0.026956,-0.714070,0.008439
4,2021,105.0,25.0,1.252343,2.291070,0.067811,0.066338,0.063567,0.762809,-0.489534,...,0.124354,1.492243,0.239900,3.081641,0.790571,0.123921,1.487050,0.234707,3.062829,0.771759


## Average of the int_reporting

In [2783]:
SG_esg_int_mean = SG_esg_int.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean['year'] = SG_esg_int_mean['year'].replace([SG_esg_int_mean['year'].mean()], 'average')
SG_esg_int_mean = pd.concat([SG_esg_int, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
3,2020,105.0,25.000000,-1.215864,-0.722509,-0.195610,-0.191964,-0.198660,-2.383923,-1.168059,...,-0.098158,-1.177893,0.037971,-0.710554,0.011954,-0.099076,-1.188908,0.026956,-0.714070,0.008439
4,2021,105.0,25.000000,1.252343,2.291070,0.067811,0.066338,0.063567,0.762809,-0.489534,...,0.124354,1.492243,0.239900,3.081641,0.790571,0.123921,1.487050,0.234707,3.062829,0.771759
5,2022,105.0,25.000000,NaN,NaN,-0.027002,-0.026119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,105.0,25.000000,NaN,NaN,0.064316,0.064435,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,105.0,24.571429,-0.133933,0.549419,-0.031383,-0.032965,-0.067191,-0.806293,-0.341095,...,0.003142,0.037706,0.171640,0.789532,0.240112,0.002022,0.024268,0.158202,0.783188,0.233769


## T-test of the int_reporting

a: an array of sample observations <br>
popmean: the expected population mean (The population mean is an average of a group characteristic.) <br>

In [2784]:
SG_res_dict = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, SG_esg_int.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=SG_esg_int.dropna()[SG_esg_int.columns[column]], popmean=0)
    temp_dict = {SG_esg_int.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict = {**SG_res_dict, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict[i]
                                    for i in SG_res_dict.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict

,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,eq_wg_r_p_y2,eq_r_alpha_y2,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
0,t-score,-0.686899,0.216696,-0.721676,-0.755934,-0.858432,-0.858432,-0.750189,0.224296,-0.035561,...,-0.380660,-0.380660,3.406262,0.474369,1.528295,-0.400842,-0.400842,3.426249,0.466503,1.505637
1,p-value,0.541480,0.842345,0.522656,0.504629,0.453770,0.453770,0.507616,0.836934,0.973866,...,0.728811,0.728811,0.042266,0.667628,0.223901,0.715377,0.715377,0.041652,0.672642,0.229235


In [2785]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df])
SG_esg_int.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
5,2022,105.0,25.000000,NaN,NaN,-0.027002,-0.026119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,105.0,25.000000,NaN,NaN,0.064316,0.064435,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,105.0,24.571429,-0.133933,0.549419,-0.031383,-0.032965,-0.067191,-0.806293,-0.341095,...,0.003142,0.037706,0.171640,0.789532,0.240112,0.002022,0.024268,0.158202,0.783188,0.233769
0,t-score,NaN,NaN,-0.686899,0.216696,-0.721676,-0.755934,-0.858432,-0.858432,-0.750189,...,-0.380660,-0.380660,3.406262,0.474369,1.528295,-0.400842,-0.400842,3.426249,0.466503,1.505637
1,p-value,NaN,NaN,0.541480,0.842345,0.522656,0.504629,0.453770,0.453770,0.507616,...,0.728811,0.728811,0.042266,0.667628,0.223901,0.715377,0.715377,0.041652,0.672642,0.229235


### Export the result into excel file

In [2786]:
sort_companies_data_top20per.to_excel(top_path, sheet_name="main", index=False)
SG_esg_int.to_excel(reporting_path, sheet_name="main", index=False)
